# Assignment 1


## Assigments 1.1

This first part of the assignments is based on the iris dataset (https://en.wikipedia.org/wiki/Iris_flower_data_set)


In [1]:
# numerical
import pandas as pd
import numpy as np

np.random.seed(42)  # set seed for reproducible results

# plotting
import plotly.express as px


# sklearn data
from sklearn.datasets import load_iris


In [2]:
iris_dataset = load_iris()

# extracting the relevant information
data = pd.DataFrame(data=iris_dataset.data, columns=iris_dataset.feature_names)
data_feature_names = data.columns
target = iris_dataset.target
target_names = iris_dataset.target_names

print(
    f"""There are {len(data_feature_names)} features, whose names are: \t{data_feature_names.values}
    \nThere are {len(target_names)} classes to predict, whose names are: \t{target_names}"
    \nThere are {data.shape[0]} observations
    """
)


There are 4 features, whose names are: 	['sepal length (cm)' 'sepal width (cm)' 'petal length (cm)'
 'petal width (cm)']
    
There are 3 classes to predict, whose names are: 	['setosa' 'versicolor' 'virginica']"
    
There are 150 observations
    


$\mathbf{Exercise\, 1.}$ Compute the median, mean and standard deviation for each of the 4 features in the iris dataset (don't use numpy or other pre-defined functions, write your own routine).


In [3]:
def mean_bh(series):
    """computes the mean of a column by hand"""
    return series.sum() / len(series)


def var_bh(series):
    """computes the variance of a column by hand"""
    series_mean = mean_bh(series)

    # variance denominator is (len(series)-1) for unbiased variance
    return ((series - series_mean) ** 2).sum() / (len(series) - 1)


def st_dev_bh(series):
    """computes the standard deviation of a column by hand"""
    series_var = var_bh(series)
    assert series_var >= 0, "variance must (and should) be positive to take its square"
    return series_var ** (1 / 2)


df_desc_bh = pd.DataFrame(columns=data.columns)
df_desc_bh.loc["mean"] = data.apply(lambda series: mean_bh(series))
df_desc_bh.loc["variance"] = data.apply(lambda series: var_bh(series))
df_desc_bh.loc["std"] = data.apply(lambda series: st_dev_bh(series))
df_desc_bh


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
mean,5.843333,3.057333,3.758000,1.199333
variance,0.685694,0.189979,3.116278,0.581006
std,0.828066,0.435866,1.765298,0.762238


We compute the values using pandas built-in functions to verify our result:


In [4]:
df_desc = pd.DataFrame(columns=data.columns)
df_desc.loc["mean"] = data.mean()
df_desc.loc["variance"] = data.var()
df_desc.loc["std"] = data.std()

df_desc


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
mean,5.843333,3.057333,3.758000,1.199333
variance,0.685694,0.189979,3.116278,0.581006
std,0.828066,0.435866,1.765298,0.762238


Due to numerical rounding errors, the values may not be exactly equal, but should be very close. We use numpy to verify that:


In [5]:
np.allclose(df_desc, df_desc_bh)


True

$\mathbf{Exercise\, 2.}$ For each feature, generate 150 Gaussian distributed samples with the same mean and standard_deviation computed in Exercise 1.


In [6]:
df_gauss = pd.DataFrame()

for col in df_desc_bh.columns:
    df_gauss[col] = np.random.normal(
        loc=df_desc_bh.loc["mean", col],  # col mean
        scale=df_desc_bh.loc["std", col],  # col std
        size=150,
    )

df_gauss


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,6.254645,3.166515,2.294577,1.151557
1,5.728841,3.208338,2.769113,1.927379
2,6.379662,2.760933,5.077196,0.447976
3,7.104503,3.158565,4.835486,1.583537
4,5.649439,3.185074,3.721102,0.795151
...,...,...,...,...
145,6.490734,2.755317,4.940172,0.810580
146,4.819056,3.449439,4.405155,0.993624
147,4.749908,3.191275,2.098832,0.453283
148,6.275535,3.411633,2.850872,0.860676


$\mathbf{Exercise\, 3.}$ Compute the histogram (with 30 bins) of the distribution of the generate samples and plot it together with the histogram of the original data. The histograms must be plotted on the same plot. Comment on the difference between simulations and real data: for which feature the distributions are most different? What is the reason?


In [141]:
# create df for data, with column indicating origin
df_comb = data.copy()
df_comb["origin"] = "data"

# create df gauss with column indicating origin
df_append = df_gauss.copy()
df_append["origin"] = "gauss"

# combine the two dataframes
df_comb = df_comb.append(other=df_append, ignore_index=True)
df_comb = pd.melt(frame=df_comb, id_vars="origin", var_name="feature") # get feature name as a column
df_comb

,origin,feature,value
0,data,sepal length (cm),5.100000
1,data,sepal length (cm),4.900000
2,data,sepal length (cm),4.700000
3,data,sepal length (cm),4.600000
4,data,sepal length (cm),5.000000
...,...,...,...
1195,gauss,petal width (cm),0.810580
1196,gauss,petal width (cm),0.993624
1197,gauss,petal width (cm),0.453283
1198,gauss,petal width (cm),0.860676


In [149]:
col = data.columns[0]
fig = px.histogram(
    data_frame=df_comb,
    barmode="overlay",
    color_discrete_sequence=px.colors.qualitative.Vivid,
    opacity=0.6,
    nbins=50,
    title="Comparison between original and synthetic (gaussian) data",
    color="origin",
    facet_col="feature",
)
# fig.update_xaxes(matches=None) # allow x axes to have different range on each subplot
fig

It appears that `petal length (cm)` and, to some extent, `petal width (cm)` are the features with the largest difference between the original data and the synthetic data.

Two analyses may be performed from this observation:
- Intuitively, our guess would be that a wider standard deviation may be responsible for such a difference. Indeed, larger standard deviation means more chance to draw far away from the mean, therefore to obtain samples substantially different samples. This explanation holds for the `petal length (cm)`, which has by far the largest standard deviation ($\approx 1.77$, whereas all the others are below $0.83$).
- However, the point above does not hold when comparing `petal width (cm)` and `sepal length (cm)`. Indeed, the former has a standard deviation of $0.76$ vs $0.83$ for the latter, but `petal width (cm)` seems more different from the original data than `sepal length (cm)`. In this case, we see two possible explanations:
    1. The true distribution of `petal width (cm)` is not gaussian than the one of `sepal length (cm)` (or at least less similar to a gaussian distribution).
    1. The sample draw from the true distribution of `petal width (cm)` is less representive than the one of `sepal length (cm)`.

$\mathbf{Exercise\, 4.}$ Estimate and plot the probability density function of the feature $\text{petal length (cm)}$.


## Assigment 1.2

$\mathbf{Exercise\, 1.}$ Create a linear benchmark, where the dimension of the input data $X$ is 2, and the dimension of the output data $y$ is 1. Choose the amount of noise and sample size.

Evaluate the prediction metrics for the following regression methods:

- Linear regression
- Support Vector Regression with RBF kernel
- Decision Tree Regression


In [2]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor


svr_rbf = SVR(kernel="rbf", C=1e3, gamma=0.1)
# fit the model by using
# y_rbf = svr_rbf.fit(X, y)
# predict by using
# svr_rbf.predict(X)


tree = DecisionTreeRegressor(max_depth=3)
# fit the model by using
# y_tree = tree.fit(X, y)
# predict by using
# tree.predict(X)


$\mathbf{Exercise\, 2.}$ Create a testing dataset and assess the testing metrics for the models.


$\mathbf{Exercise\, 3.}$ Plot the regression results (training and testing) for each model


$\mathbf{Exercise\, 4.}$ Create a classification benchmark with 3 labels.

Evaluate training and testing prediction metrics for the following classification methods:

- K-nearest neighbours
- Random Forest
- Naive Bayes


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
